In [ ]:

%%python -m pip install pymongo
!pip install sqlalchemy


In [ ]:

import os
import pandas as pd
from sqlalchemy import create_engine
import pymongo


In [ ]:

# MySQL connection details
mysql_args = {
    "user": "root",
    "password": os.getenv("MYSQL_PASSWORD"),  # Use environment variable
    "host": "localhost",
    "dbname": "grocery_inventory_db"
}


In [ ]:

# Connect to MongoDB (local)
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client["dim_date_product_db"]

# Specify collection
collection = db["dim_date_product"]
query = {}  # Select all records
df_mongo = pd.DataFrame(list(collection.find(query)))

# Display the first 5 rows of MongoDB data
df_mongo.head()


In [ ]:

# Create MySQL connection
conn_str = f"mysql+pymysql://{mysql_args['user']}:{mysql_args['password']}@{mysql_args['host']}/{mysql_args['dbname']}"
sqlEngine = create_engine(conn_str)

# Insert data from MongoDB DataFrame into MySQL
table_name = "dim_date_product"
df_mongo.to_sql(name=table_name, con=sqlEngine, if_exists="replace", index=False)
print(f"✅ Data successfully inserted into `{table_name}` table in `{mysql_args['dbname']}` database.")


In [ ]:

# Verify the data inserted into MySQL
sql_query = "SELECT * FROM dim_date_product LIMIT 5;"
df_mysql = pd.read_sql(sql_query, con=sqlEngine)

# Display the first 5 rows from MySQL
df_mysql.head()
